In [1]:
import pandas as pd
import numpy as np
import os
import sys
import re
import time
import gc

from matplotlib import pyplot as plt
from dotenv import load_dotenv

sys.path.append("../")

from scripts import search
from scripts import user_requests as u_reqs
from scripts import rendering
from models.meteorite import Meteorite


load_dotenv()
plt.style.use('Solarize_Light2')

# Setting default DPI, pulling it from dotenv if it exists, setting it on 100 if not

try:
    pc_dpi = int(os.getenv('DPI'))
except TypeError:
    pc_dpi = 100
if pc_dpi is None:
    pc_dpi = 100


/var/folders/3s/s8sp6jwn6qs02jfxbgjc7c_40000gn/T/ipykernel_23168/334325237.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df_meteorites = pd.read_pickle(filepath_or_buffer="../datasets/metbull_data.pkl")


In [3]:
display(df_meteorites.head(n=20))


,name,year,country,type,mass,URL,numeric_id
0,Denader 001,2022,Mali,H4-melt breccia,5330.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,1
1,Hassi Khebi 001,2022,Algeria,C3-ung,500.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,1
2,Qaen 001,2016,Iran,L6,21000.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,1
3,Aachen,1880,Germany,L5,21.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
4,Aammiq,2000,Lebanon,H6,596.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
5,Aarhus,1951,Denmark,H6,720.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
6,Aba,1927,Japan,Pseudometeorite,0.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
7,Aba Panu,2018,Nigeria,L3,160000.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
8,Abadla,2015,Algeria,Eucrite-cm,222.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
9,Abadla 002,2022,Algeria,CM2,800.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,2


# <u> Use cases :</u>

## <u> Name :</u>
### <u>case 1 : exact match : lets say I want Catalina 20</u>

In [4]:
result = search.search_by_name(df=df_meteorites, name_query="Catalina", numeric_range=20)
display(result)


,name,year,country,type,mass,URL,numeric_id
7820,Catalina 020,2010,Chile,L6,2080.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,20


### case 2 : lets say I want Yamato 100 -> 150 :

In [5]:
# case insensitive, so I can put caps wherever I want it doesnt matter

result = search.search_by_name(df=df_meteorites, name_query="YaMaTO", numeric_range=(100, 150))
display(result.head(n=5))
display(result.tail(n=5))


,name,year,country,type,mass,URL,numeric_id
77577,Yamato 000100,2000,Antarctica,<NA>,0.20,https://www.lpi.usra.edu/meteor/metbull.php?co...,100
77578,Yamato 000101,2000,Antarctica,<NA>,0.74,https://www.lpi.usra.edu/meteor/metbull.php?co...,101
77579,Yamato 000102,2000,Antarctica,H5-6,12.50,https://www.lpi.usra.edu/meteor/metbull.php?co...,102
77580,Yamato 000103,2000,Antarctica,<NA>,3.31,https://www.lpi.usra.edu/meteor/metbull.php?co...,103
77581,Yamato 000104,2000,Antarctica,<NA>,1.49,https://www.lpi.usra.edu/meteor/metbull.php?co...,104


,name,year,country,type,mass,URL,numeric_id
77623,Yamato 000146,2000,Antarctica,<NA>,1.06,https://www.lpi.usra.edu/meteor/metbull.php?co...,146
77624,Yamato 000147,2000,Antarctica,<NA>,2.83,https://www.lpi.usra.edu/meteor/metbull.php?co...,147
77625,Yamato 000148,2000,Antarctica,<NA>,1.38,https://www.lpi.usra.edu/meteor/metbull.php?co...,148
77626,Yamato 000149,2000,Antarctica,H4,84.60,https://www.lpi.usra.edu/meteor/metbull.php?co...,149
77627,Yamato 000150,2000,Antarctica,L5,142.20,https://www.lpi.usra.edu/meteor/metbull.php?co...,150


## <u>Types :</u>

In [6]:
# Pretty straighforward, case insensitive, as scrapped on metbull :

result = search.search_by_type(df=df_meteorites, query="l3")
print(result.__len__())
display(result.head(n=5))


834


,name,year,country,type,mass,URL,numeric_id
7,Aba Panu,2018,Nigeria,L3,160000.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
409,Acfer 341,2002,Algeria,L3,132.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,341
415,Acfer 347,2001,Algeria,L3,1165.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,347
635,Al Huqf 068,2009,Oman,L3,165.4,https://www.lpi.usra.edu/meteor/metbull.php?co...,68
723,Alatage Mountain 049,2023,China,L3,568.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,49


## <u>Country :</u>

In [7]:
# pretty much the same :

result = search.search_by_country(df=df_meteorites, query="France")
print(result.__len__())
display(result.head(n=5))


90


,name,year,country,type,mass,URL,numeric_id
62,Abbans-Dessous,2015,France,LL6,3.12,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
520,Agen,1814,France,H5,30000.00,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
562,Aire-sur-la-Lys,1769,France,<NA>,6800.00,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
670,Alais,1806,France,CI1,6000.00,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
735,Alby sur Chéran,2002,France,Eucrite-mmict,252.00,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>


### Pretty basic but it gets the job done.

# <u>Now let's try to join a few meteorites in a single dataframe to request further details via MetBull :</u>

In [8]:
group_1 = search.search_by_name(df=df_meteorites, name_query="catalina", numeric_range=(30, 60))

# We can also stack the searches, so here, lets take yamato 1000-1100, type h4 :
group_2 = search.search_by_type(
    df=search.search_by_name(df=df_meteorites, name_query="yamato", numeric_range=(1000, 1100)),
    query="h4"
    )

group_3 = search.search_by_country(df=df_meteorites, query="lebanon")


In [9]:
# Concat the result :
df_search = pd.concat([group_1, group_2, group_3]).drop_duplicates().reset_index(drop=True)

display(df_search)


,name,year,country,type,mass,URL,numeric_id
0,Catalina 030,2010,Chile,H5,214.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,30
1,Catalina 031,2010,Chile,L6,1178.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,31
2,Catalina 032,2010,Chile,H4,1107.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,32
3,Catalina 033,2010,Chile,L6,211.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,33
4,Catalina 034,2010,Chile,LL5,20.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,34
...,...,...,...,...,...,...,...
58,Yamato 001091,2000,Antarctica,H4,6.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,1091
59,Yamato 001095,2000,Antarctica,H4,7.5,https://www.lpi.usra.edu/meteor/metbull.php?co...,1095
60,Aammiq,2000,Lebanon,H6,596.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>
61,Aphaca,<NA>,Lebanon,Doubtful meteorite,0.0,https://www.lpi.usra.edu/meteor/metbull.php?co...,<NA>


# <u>Okay we got our meteorite sample, now let's get this done:</u>
- Load the selected meteorites as objects of the Meteorite class
- Launch the scrapper
- Get a coffee (the server is deal slow and rate is limited to 25RPS to avoid spam)
- Hopefully get the results

In [10]:
meteorite_objects = u_reqs.make_meteorites(selected_meteorites=df_search)

# Checking
print(meteorite_objects[0])
print(meteorite_objects[1])
print(meteorite_objects[20])
print(meteorite_objects[30])
print(meteorite_objects[-2])
print(meteorite_objects[-1])


Catalina 030
Catalina 031
Catalina 050
Catalina 060
Aphaca
Beyrout


In [11]:
tzero = time.perf_counter()

u_reqs.request_selected(meteorites=meteorite_objects)

tf = time.perf_counter()

print(f"Processed in {round(tf - tzero, ndigits=4)}")


Processing meteorites: 100%|██████████| 63/63 [00:42<00:00,  1.47it/s]

The table couldnt be foundThe table couldnt be found
You can find more informations for this meteorite here : https://www.lpi.usra.edu/meteor/metbull.php?code=5035

You can find more informations for this meteorite here : https://www.lpi.usra.edu/meteor/metbull.php?code=2316
Processed in 43.0041


### Okay so it looks like two have no info attached but the script returned the link

In [12]:
df_rend = rendering.show_properties(meteorite_list=meteorite_objects, omit=[])

df_rend.head(n=30)


Duds : 2


,type,coordinates,latitude,longitude,fall_country,weathering_g,shock_stage,mag_sus,fs_content,wo_content,fa_content,tsm,pieces,type_spec_loc
name,,,,,,,,,,,,,,
Catalina 030,H5,"(-25.233333333333334, -69.71666666666667)",~25°14'S,~69°43'W,Chile,W2,None,5.01,NaN,NaN,NaN,52,2,CEREGE
Catalina 031,L6,"(-25.233333333333334, -69.71666666666667)",~25°14'S,~69°43'W,Chile,W2,None,4.59,NaN,NaN,NaN,51,17,CEREGE
Catalina 032,H4,"(-25.233333333333334, -69.71666666666667)",~25°14'S,~69°43'W,Chile,W2,None,5.08,17.2,1.4,19.4,57,12,CEREGE
Catalina 033,L6,"(-25.233333333333334, -69.71666666666667)",~25°14'S,~69°43'W,Chile,W3,None,4.25,NaN,NaN,NaN,31,1,CEREGE
Catalina 034,LL5,"(-25.233333333333334, -69.71666666666667)",~25°14'S,~69°43'W,Chile,W4,None,4.05,22.5,1.3,26.8,6,1,CEREGE
Catalina 035,H5,"(-25.233333333333334, -69.71666666666667)",~25°14'S,~69°43'W,Chile,W1,None,5.17,NaN,NaN,NaN,21,1,CEREGE
Catalina 036,H5,"(-25.233333333333334, -69.71666666666667)",~25°14'S,~69°43'W,Chile,W1,None,5.22,NaN,NaN,NaN,10,1,CEREGE
Catalina 037,Ureilite,"(-25.1, -69.75833)",25.1°S,69.75833°W,Chile,medium,S1,NaN,16.6,10.9,19.9,83.5,>100,ASU
Catalina 038,H5,"(-25.233333333333334, -69.71666666666667)",~25°14'S,~69°43'W,Chile,W1,None,5.11,NaN,NaN,NaN,29,6,CEREGE


In [13]:
df_rend.dtypes


type              object
coordinates       object
latitude          object
longitude         object
fall_country      object
weathering_g      object
shock_stage       object
mag_sus           object
fs_content       float64
wo_content       float64
fa_content       float64
tsm               object
pieces            object
type_spec_loc     object
dtype: object

In [14]:
# clean up : 

[met.purge_html() for met in  meteorite_objects]

gc.collect()


0

## Lets do a random selection of ... lets say 200 meteorites, look at the rendering if its doable, and then grab a cold drink

In [15]:
df_random = df_meteorites.dropna(subset=["mass", "type"]).sample(n=200, random_state=123).reset_index(drop=True)


In [16]:
display(df_random.head(n=5))
display(df_random.tail(n=5))


,name,year,country,type,mass,URL,numeric_id
0,Grove Mountains 091409,2010,Antarctica,L5,30.00,https://www.lpi.usra.edu/meteor/metbull.php?co...,91409
1,Queen Alexandra Range 94643,1994,Antarctica,L6,11.10,https://www.lpi.usra.edu/meteor/metbull.php?co...,94643
2,Roberts Massif 04303,2004,Antarctica,L5,16.60,https://www.lpi.usra.edu/meteor/metbull.php?co...,4303
3,Grove Mountains 020351,2003,Antarctica,L6,0.64,https://www.lpi.usra.edu/meteor/metbull.php?co...,20351
4,Lewis Cliff 86010,1986,Antarctica,Angrite,6.90,https://www.lpi.usra.edu/meteor/metbull.php?co...,86010


,name,year,country,type,mass,URL,numeric_id
195,Grove Mountains 090120,2010,Antarctica,L5,0.79,https://www.lpi.usra.edu/meteor/metbull.php?co...,90120
196,Thiel Mountains 99017,2000,Antarctica,L4,1528.00,https://www.lpi.usra.edu/meteor/metbull.php?co...,99017
197,Yamato 792400,1979,Antarctica,Stone-uncl,1.38,https://www.lpi.usra.edu/meteor/metbull.php?co...,792400
198,Miller Range 05058,2005,Antarctica,L6,128.10,https://www.lpi.usra.edu/meteor/metbull.php?co...,5058
199,Meteorite Hills 01227,2001,Antarctica,L5,17.10,https://www.lpi.usra.edu/meteor/metbull.php?co...,1227


In [17]:
# okay hopefully there are a lot of info and just a few with no tables
# and the server loves me

random_meteorites = u_reqs.make_meteorites(selected_meteorites=df_random)


In [18]:
u_reqs.request_selected(random_meteorites)


Processing meteorites:  12%|█▏        | 24/200 [00:28<04:34,  1.56s/it]

The table couldnt be found
You can find more informations for this meteorite here : https://www.lpi.usra.edu/meteor/metbull.php?code=16958


Processing meteorites:  18%|█▊        | 36/200 [00:36<01:34,  1.74it/s]

The table couldnt be found
You can find more informations for this meteorite here : https://www.lpi.usra.edu/meteor/metbull.php?code=29067


Processing meteorites:  20%|██        | 41/200 [00:36<00:38,  4.09it/s]

...

The table couldnt be found
You can find more informations for this meteorite here : https://www.lpi.usra.edu/meteor/metbull.php?code=36979


Processing meteorites:  99%|█████████▉| 198/200 [02:30<00:00,  2.70it/s]

The table couldnt be found
You can find more informations for this meteorite here : https://www.lpi.usra.edu/meteor/metbull.php?code=4007
The table couldnt be found
You can find more informations for this meteorite here : https://www.lpi.usra.edu/meteor/metbull.php?code=27749


Processing meteorites: 100%|██████████| 200/200 [02:30<00:00,  1.32it/s]

The table couldnt be found
You can find more informations for this meteorite here : https://www.lpi.usra.edu/meteor/metbull.php?code=28044


In [19]:
# Looks like we have a lot of duds, we can count em : itll be the ones with table_soup = None
duds = 0
for met in random_meteorites:
    if met.table_soup is None:
        duds += 1

print(f"And we have {duds} duds")


And we have 31 duds


In [20]:
# clean up a bit :

for met in random_meteorites:
    met.purge_html()

gc.collect()


0

#### all in all its pretty good for a random sample, all right lets see what we have

In [21]:
render_random = rendering.show_properties(meteorite_list=random_meteorites, omit=["pieces", "type_spec_loc"])
gc.collect()


Duds : 200


0

In [22]:
# ok so we have 200 meteorites here :

display(render_random.head(n=5))
display(render_random.tail(n=5))

,type,coordinates,latitude,longitude,fall_country,weathering_g,shock_stage,mag_sus,fs_content,wo_content,fa_content,tsm
name,,,,,,,,,,,,
Grove Mountains 091409,L5,"(-72.98366666666666, 75.24766666666667)","72°59'1.2""S","75°14'51.6""E",Antarctica,W2,S2,NaN,20.1,NaN,24.1,0.03
Queen Alexandra Range 94643,L6,"(nan, nan)",NaN,NaN,Antarctica,B/C,None,NaN,NaN,NaN,NaN,NaN
Roberts Massif 04303,L5,"(nan, nan)",NaN,NaN,Antarctica,C,None,NaN,NaN,NaN,NaN,NaN
Grove Mountains 020351,L6,"(nan, nan)","72°59'02""S","75°14'47""E",Antarctica,W1,S1,NaN,20.6,1.4,24.4,NaN
Lewis Cliff 86010,Angrite,"(nan, nan)",NaN,NaN,Antarctica,A/B,None,NaN,19.0,NaN,63.0,NaN


,type,coordinates,latitude,longitude,fall_country,weathering_g,shock_stage,mag_sus,fs_content,wo_content,fa_content,tsm
name,,,,,,,,,,,,
Grove Mountains 090120,L5,"(-72.77925, 75.32319444444444)","72°46'45.3""S","75°19'23.5""E",Antarctica,W2,S3,NaN,21.3,1.6,25.1,0.79
Thiel Mountains 99017,L4,"(nan, nan)",85°09.807'S,94°51.417'W,Antarctica,A,S3,NaN,21.1,NaN,24.2,NaN
Yamato 792400,Stoneuncl,"(nan, nan)",NaN,NaN,Antarctica,None,None,NaN,NaN,NaN,NaN,NaN
Miller Range 05058,L6,"(nan, nan)",NaN,NaN,Antarctica,A/B,None,NaN,NaN,NaN,NaN,NaN
Meteorite Hills 01227,L5,"(nan, nan)",NaN,NaN,Antarctica,B,None,NaN,NaN,NaN,NaN,NaN


In [23]:
# looks like we have some dms that failed to convert

df_conversion_failures = render_random[
    (render_random["coordinates"] == (np.nan, np.nan))
    &
    ((render_random["latitude"].notna() & render_random["longitude"].notna()))
    ]


In [24]:
df_conversion_failures.head()


,type,coordinates,latitude,longitude,fall_country,weathering_g,shock_stage,mag_sus,fs_content,wo_content,fa_content,tsm
name,,,,,,,,,,,,
Grove Mountains 020351,L6,"(nan, nan)","72°59'02""S","75°14'47""E",Antarctica,W1,S1,NaN,20.6,1.4,24.4,NaN
Grove Mountains 022777,L6,"(nan, nan)","72°46'30""S","75°19'47""E",Antarctica,W1,S4,NaN,20.6,1.4,24.2,NaN
Grove Mountains 053498,L6,"(nan, nan)","72°46'44""S","75°19'00""E",Antarctica,W1,S3,NaN,20.3,1.7,23.6,NaN
Grove Mountains 051612,H5,"(nan, nan)","72°49'46""S","75°16'43""E",Antarctica,W2,S2,NaN,17.2,1.4,19.7,NaN
Grove Mountains 054458,H6,"(nan, nan)","72°54'57""S","75°06'03""E",Antarctica,W1,S1,NaN,16.9,1.6,19.5,NaN


In [25]:
# Lets save the conversion failure dataset for further examination :

df_conversion_failures.to_csv(path_or_buf="../datasets/conversion_failures.csv")
df_conversion_failures.to_pickle(path="../datasets/conversion_failures.pkl")
